In [1]:
import os
import numpy as np
import cv2
import math

def ci95(inp):
    max95 = np.mean(inp) + (1.96 * (np.std(inp) / math.sqrt(len(inp))))
    min95 = np.mean(inp) - (1.96 * (np.std(inp) / math.sqrt(len(inp))))
    return min95, max95

name_li = ['brmh','kyuh','gugh']
for name in name_li:
    print(name)
    gt_base = 'D:/lunit/01_data_analysis/data/img/result/{}_healthcheck/gt_mask'.format(name)
    base = 'D:/lunit/01_data_analysis/data/img/ai_json(all_files)/{}_healthcheck/ai_mask'.format(name)
    ai_mask_list = os.listdir(base)
    sum_image = len(ai_mask_list)

    fppi_cnt = 0
    fn_cnt = 0
    tp_cnt = 0
    
    total_cnt=0
    gt_ims= []
    ai_ims = []
    true_detection_rate_li = []
    for index, ai_mask in enumerate(sorted(ai_mask_list)):
#         if index==10:
#             break
        tp = 0
        fn = 0
        gt_file = '{}_mask.png'.format(ai_mask.split('_cutoff')[0])
        gt_im = cv2.imread(os.path.join(gt_base, gt_file))
        w,h,_=gt_im.shape
        im = cv2.imread(os.path.join(base, ai_mask))
        im = cv2.resize(im, (h,w),interpolation=cv2.INTER_NEAREST)
#         print(gt_file, ai_mask)
        
        imgray = cv2.cvtColor(gt_im,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
        gt_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
        ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#         print(len(gt_contours), len(ai_contours))
        total_cnt+=len(ai_contours)
    #     print(gt_im.shape, im.shape)
        if gt_im.sum() == 0: # normal
            fppi_cnt += len(ai_contours)
            
        elif im.sum()==0:
            fn_cnt += len(gt_contours)
            fn+=1
            
        else:
            for ai_contour in ai_contours:
                _image = np.zeros_like(gt_im)
                _image = cv2.drawContours(_image, [ai_contour], 0, color=255, thickness=-1)
                if _image[gt_im>0].sum() == 0:
                    fppi_cnt += 1
                
                    
            for gt_contour in gt_contours:
                _image = np.zeros_like(gt_im)
                _image = cv2.drawContours(_image, [gt_contour], 0, color=255, thickness=-1)
                if _image[im>0].sum() == 0:
                    fn_cnt += 1
                    fn+=1
                else:
                    tp_cnt+=1
                    tp+=1
        
        if tp+fn>0:
            true_detection_rate = tp/(tp+fn)
            true_detection_rate_li.append(true_detection_rate)


    fppi = fppi_cnt/sum_image
    print('Calculated FPPI: {:0.2f}'.format(fppi))
    
    true_detection_rate = tp_cnt/(tp_cnt+fn_cnt)
    min_ci, max_ci = ci95(np.array(true_detection_rate_li))
    print('true_detection_rate: {:0.2f} ({:0.2f}, {:0.2f})'.format(true_detection_rate, min_ci, max_ci))


brmh
Calculated FPPI: 0.35
true_detection_rate: 0.42 (0.29, 0.45)
kyuh
Calculated FPPI: 0.18
true_detection_rate: 0.45 (0.36, 0.46)
gugh
Calculated FPPI: 0.67
true_detection_rate: 0.54 (0.46, 0.57)


In [11]:
def ci95(inp):
    max95 = np.mean(inp) + (1.96 * (np.std(inp) / math.sqrt(len(inp))))
    min95 = np.mean(inp) - (1.96 * (np.std(inp) / math.sqrt(len(inp))))
    return min95, max95

min_ci, max_ci = ci95(np.array(true_detection_rate_li))
print(min_ci, true_detection_rate , max_ci)

0.4967278800040549 0.425 0.5055844702603889


In [ ]:
import os
import numpy as np
import cv2
import math

gt_base = 'D:/lunit/data/img/result/brmh_respiratory/gt_mask'
base = 'D:/lunit/data/img/ai_json(all_files)/brmh_respiratory/ai_mask'
ai_mask_list = os.listdir(base)
sum_image = len(ai_mask_list)

fppi_cnt = 0
gt_ims= []
ai_ims = []
fppi_score = []
for index, ai_mask in enumerate(sorted(ai_mask_list)):
    # if index==10:
    #     break

    gt_file = '{}_mask.png'.format(ai_mask.split('_cutoff')[0])
    gt_im = cv2.imread(os.path.join(gt_base, gt_file), cv2.IMREAD_ANYDEPTH | cv2.IMREAD_GRAYSCALE)
#     gt_ims.append(gt_im)
    w,h=gt_im.shape
#     print(h,w)
#     print("started reading gt file: {}".format(gt_file))

    im = cv2.imread(os.path.join(base, ai_mask))
    im = cv2.resize(im, (h,w),interpolation=cv2.INTER_NEAREST)
#     ai_ims.append(im)
    imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
    ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     print(gt_im.shape, im.shape)
    fppi_cnt_single = 0
    if gt_im.sum() == 0: # normal
        fppi_cnt += len(ai_contours)
        fppi_cnt_single += len(ai_contours)
        
    else:
        for ai_contour in ai_contours:
            _image = np.zeros_like(gt_im)
            _image = cv2.drawContours(_image, [ai_contour], 0, color=255, thickness=-1)
            if _image[gt_im>0].sum() == 0:
                fppi_cnt += 1
                fppi_cnt_single += len(ai_contours)
    
    fppi_score.append(fppi_cnt_single/1)
#     print("intermediate sum ffpi cnt: {}".format(fppi_cnt))
    
    

fppi = fppi_cnt/sum_image
print('Calculated FPPI: {}'.format(fppi))

fppi_score = np.array(fppi_score)
mean = fppi_score.mean()
std = fppi_score.std()
ci = 1.96*(std/math.sqrt(sum_image))

confidence_lower = mean-ci
confidence_upper = mean+ci
print("FPPI = {:0.4f} [{:0.3f} - {:0.3}]".format(fppi, confidence_lower, confidence_upper))

# FPPI 계산 (CI 구간 X)

In [5]:
import os
import numpy as np
import cv2

names = ['brmh_healthcheck','kyuh_healthcheck','gugh_healthcheck']
for name in names:
    gt_base = 'D:/lunit/data/img/result/'+name+'/gt_mask'
    base = 'D:/lunit/data/img/ai_json(all_files)/'+name+'/ai_mask'
    ai_mask_list = os.listdir(base)
    sum_image = len(ai_mask_list)

    fppi_cnt = 0
    for index, ai_mask in enumerate(sorted(ai_mask_list)):
        # if index==10:
        #     break

        gt_file = '{}_mask.png'.format(ai_mask.split('_cutoff')[0])
        gt_im = cv2.imread(os.path.join(gt_base, gt_file), cv2.IMREAD_ANYDEPTH | cv2.IMREAD_GRAYSCALE)
        w,h=gt_im.shape
    #     print(h,w)
    #     print("started reading gt file: {}".format(gt_file))

        im = cv2.imread(os.path.join(base, ai_mask))
        im = cv2.resize(im, (h,w))
        imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
        ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        if gt_im.sum() == 0: # normal
            fppi_cnt += len(ai_contours)

        else:
            for ai_contour in ai_contours:
                _image = np.zeros_like(gt_im)
                _image = cv2.drawContours(_image, [ai_contour], 0, color=255, thickness=-1)
                if _image[gt_im>0].sum() == 0:
                    fppi_cnt += 1
    #     print("intermediate sum ffpi cnt: {}".format(fppi_cnt))

    print(name)
    print('Calculated FPPI: {}'.format(fppi_cnt/sum_image))

brmh_healthcheck
Calculated FPPI: 0.3547815820543093
kyuh_healthcheck
Calculated FPPI: 0.17987152034261242
gugh_healthcheck
Calculated FPPI: 0.6666666666666666
